In [37]:
import json
import pandas as pd
import requests

now_playing = []
for i in range(1,1000):
    url = 'https://api.themoviedb.org/3/movie/now_playing?api_key={1}&language=ko-KR&page={0}'.format(i,'5ecfbcb3280f5e9f7898c3ca00acca81')
    resp = requests.get(url)
    now_playing.append(json.loads(resp.content))

    if i == 10:
        break

now_playing_movies = []
for row in now_playing:
    for result in row ['results']:
        now_playing_movies.append(result)

In [38]:
now_playing_movies=pd.DataFrame(now_playing_movies)
now_playing_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              200 non-null    bool   
 1   backdrop_path      200 non-null    object 
 2   genre_ids          200 non-null    object 
 3   id                 200 non-null    int64  
 4   original_language  200 non-null    object 
 5   original_title     200 non-null    object 
 6   overview           200 non-null    object 
 7   popularity         200 non-null    float64
 8   poster_path        200 non-null    object 
 9   release_date       200 non-null    object 
 10  title              200 non-null    object 
 11  video              200 non-null    bool   
 12  vote_average       200 non-null    float64
 13  vote_count         200 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 19.3+ KB


In [39]:
now_playing_movies.to_csv("../data-files/now_playing_movies.csv", header=True, index=True)

In [5]:
now_playing_movies=now_playing_movies.drop(['adult', 'backdrop_path', 'video','genre_ids'], axis=1)

In [6]:
now_playing_movies.head(2)

,id,original_language,original_title,overview,popularity,poster_path,release_date,title,vote_average,vote_count
0,580489,en,Venom: Let There Be Carnage,연쇄살인마 클리터스 캐서디(우디 해럴슨)가 사형선고를 받는다. 죽을 생각이 전혀 없...,11473.714,/1Lh9LER4xRQ3INFFi2dfS2hpRwv.jpg,2021-09-30,베놈 2: 렛 데어 비 카니지,7.2,3947
1,512195,en,Red Notice,인터폴이 중범죄 피의자를 대상으로 내리는 전 세계적 경보인 레드 노티스. 세계 곳곳...,3991.047,/gG06Uu6UQseYaPx1RU2PQjFEUKK.jpg,2021-11-04,레드 노티스,6.9,1716


In [7]:
# 칼럼이름을 DB 칼럼이름과 동일하게 변경

now_playing_movies=now_playing_movies.rename(columns={'id':'movie_id', 'original_title':'subtitle','poster_path':'posterpath'})
now_playing_movies.head(2)

,movie_id,original_language,subtitle,overview,popularity,posterpath,release_date,title,vote_average,vote_count
0,580489,en,Venom: Let There Be Carnage,연쇄살인마 클리터스 캐서디(우디 해럴슨)가 사형선고를 받는다. 죽을 생각이 전혀 없...,11473.714,/1Lh9LER4xRQ3INFFi2dfS2hpRwv.jpg,2021-09-30,베놈 2: 렛 데어 비 카니지,7.2,3947
1,512195,en,Red Notice,인터폴이 중범죄 피의자를 대상으로 내리는 전 세계적 경보인 레드 노티스. 세계 곳곳...,3991.047,/gG06Uu6UQseYaPx1RU2PQjFEUKK.jpg,2021-11-04,레드 노티스,6.9,1716


In [8]:
now_playing_movies.isna().sum()

movie_id             0
original_language    0
subtitle             0
overview             0
popularity           0
posterpath           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [9]:
now_playing_movies_ids=now_playing_movies[["movie_id"]]
now_playing_movies_ids

,movie_id
0,580489
1,512195
2,585245
3,566525
4,589761
...,...
195,447200
196,588099
197,424277
198,712121


In [48]:
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()
error_list = []
cursor.execute('DELETE FROM actor')
cursor.execute('DELETE FROM now_playing_movies')
for movie_id in now_playing_movies_ids.values:
    try:
        # print(type(movie_id), movie_id)  # movie_id type이 nd.arrary임. 
        sql= "INSERT INTO now_playing_movies VALUES(%s)"
        cursor.execute(sql, movie_id[0])
    except:
        error_list.append(movie_id)
    
conn.commit()
cursor.close()
conn.close()

In [40]:
#없는 영화 업데이트

popular_movies = []
for i in error_list:
        for e in i:
                url="https://api.themoviedb.org/3/movie/{0}?api_key={1}&language=ko-KR".format(e,'36927ad1d2817ff0fa31947e47d186fd')
                response = requests.get(url)
                popular_movies.append(json.loads(response.content))

In [43]:
movies2 = pd.DataFrame(popular_movies)

In [45]:
# 3. DB에 저장 ( "resource you requested could not be found."행 영화는 제거하고 필요한 컬럼만 가져온 후 저장)
import numpy as np
# 1. connect
conn = pymysql.connect(host="127.0.0.1", port=3306, db='finalteam3', user="kdigital", password="mysql", charset="utf8")

# 2. get command object
cursor = conn.cursor()

# cursor.execute("delete from movie") #기존 데이터를 지울 때 주석을 풀어주세요 

error_list = []   ##결과 18개
            
# movies2['original_language'] = movies2['original_language'].apply(lambda ol: ol if ol else '')
movies2['overview'] = movies2['overview'].apply(lambda ov : ov if str(ov) != 'nan' else '')
movies2['release_date'] = movies2['release_date'].apply(lambda ov : ov if str(ov) != 'nan' else '')
movies2['poster_path'] = movies2['poster_path'].apply(lambda ov : ov if str(ov) != 'nan' else '')
print(movies2.values.shape)
##결과: 630186 entries  ???????????????

# 3. execute command
sql = """insert into movie (movie_id, title, subtitle, original_language, overview, popularity, release_date, vote_average, vote_count, posterpath) 
        values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
for movie in movies2.values:
    cursor.execute(sql, list(movie[list(np.array([9, 24, 12, 11, 13, 14, 18, 26, 27, 15]) - 3)]))

# movie3.append(list(movies2.values)) ##결과: 467108 entries    ????????????????????
                

conn.commit() # confirm previous execution
# 4. close resource
cursor.close()
conn.close()

(34, 25)
